# Preparazione

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.7 MB/s eta 0:00:00


In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.562s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=5442ffb3ebb2ac28a731c9ed641f3078dca316dfd0d9b52071d2473ab4409b07
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Demo

In [90]:
%%writefile app.py
import streamlit as st
import pyspark
import pandas as pd
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALSModel

conf = SparkConf().setAppName("BDC_Demo").setMaster("local")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

st.title('Videogame Recommender System')

als_model = ALSModel.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/models/ALS10_08_2023__13_28_08')
demo_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_data',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
demo_cluster = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_cluster',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
demo_game_cluster = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_game_cluster_data',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")
games_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/games.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

def compute_ALS_recommendation(steam_id):
  st.write('Computing the recommendations based on the item you already liked...')
  steam_int_id = demo_df.filter(demo_df.steam_id == steam_id).first()['steam_int_id']
  user = spark.createDataFrame([steam_int_id], "int").toDF("steam_int_id")
  user_recs = als_model.recommendForUserSubset(user, 5)
  display_ALS_recommendation(user_recs)

def display_ALS_recommendation(recommendation_df, games_info=games_df):
  st.write('Displaying the information about the recommendations computed...')
  data = recommendation_df.first()
  recommendations_value = data["recommendations"]
  recommendations = [{row.steam_appid : row.rating} for row in recommendations_value]
  recommendations_info = []
  for recommendation in recommendations:
    data = []
    appid = list(recommendation.keys())[0]
    title = games_info.filter(games_info.steam_appid == appid).first()['name']
    data.append(title)
    value = recommendation[appid]
    data.append(value)
    description = games_info.filter(games_info.steam_appid == appid).first()['short_description']
    data.append(description)
    link = 'https://store.steampowered.com/app/{}'.format(appid)
    data.append(link)
    recommendations_info.append(data)
  columns = ['Title', 'Rating', 'Description', 'Steam Page']
  df = pd.DataFrame(recommendations_info, columns=columns)
  st.table(df)

def compute_cluster_recommendation(title):
  cluster = demo_cluster.filter(demo_cluster.name == title).first()['cluster']
  games_in_cluster = demo_cluster.filter(sqlf.col('cluster') == cluster)
  recommendations = games_in_cluster.sample(False, fraction=5/games_in_cluster.count())
  display_cluster_recommendation(recommendations)

def display_cluster_recommendation(recommendation_df, games_info=games_df):
  recommendations_info = []
  recommendations = recommendation_df.collect()
  for recommendation in recommendations:
    data = []
    title = recommendation['name']
    data.append(title)
    appid = recommendation['steam_appid']
    description = games_info.filter(games_info.steam_appid == appid).first()['short_description']
    data.append(description)
    link = 'https://store.steampowered.com/app/{}'.format(appid)
    data.append(link)
    recommendations_info.append(data)
  columns = ['Title', 'Description', 'Steam Page']
  df = pd.DataFrame(recommendations_info, columns=columns)
  st.table(df)


st.write("What do you care about the most?\n Others' opinions or the story that you are going to live?")
choice = st.selectbox('Select your choice', options=["", "Others' opinions", "The story to live"])

if choice == "Others' opinions":
  id_list = [row.steam_id for row in demo_df.select('steam_id').distinct().collect()]
  steam_id = st.selectbox('Select the Steam ID', options=['', *id_list])
  btn_als = st.button("Compute Recommendations!")
  if btn_als:
    compute_ALS_recommendation(steam_id)
elif choice == "The story to live":
  game_list = [row.name for row in demo_game_cluster.select('name').distinct().collect()]
  game = st.selectbox('Select one game you liked', options=['', *game_list])
  btn_cluster = st.button("Compute Recommendations!")
  if btn_cluster:
    compute_cluster_recommendation(game)
else:
  st.write("Please select one of the possible options")



Overwriting app.py


In [91]:
!streamlit run app.py &>/content/logs.txt &

In [92]:
!curl ipv4.icanhazip.com

34.74.39.230


In [93]:
!npx localtunnel --port 8501

npx: installed 22 in 2.205s
your url is: https://some-trams-notice.loca.lt
^C
